In [1]:
import json
import pandas as pd
from pathlib import Path
from cassis import *
from cassis.xmi import CasXmiSerializer

In [2]:
OUTPUT_DIR = Path("/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject")

# готовим неразмеченные данные

In [3]:
descriptions = {
    "y": ["простая команда (без атрибутов)", "движение в направлении", "движение на кол-во метров",
          "движение на кол-во градусов", "движение на кол-во часов", "движение к объектам", 
          "движение к близкому объекту", "движение с 1 отношением", "движение с 2 отношениями", 
          "патрулирование по кругу", "движение к объектам относительно робота", "движение по взгляду"],
    "action": ["патрулировать", "остановиться", "двигаться в направлении", "поворот в направлениие",
               "двигаться на заданное расстояние", "поворачивать на заданный угол", "двигаться к объекту",
               "искать", "обходить", "осматривать", "поворачивать к объекту", "прерваться", "продолжать", "анализ"],
    "direction": ["", "вперед", "назад", "направо", "налево", "на север", "на юг", "на восток", "на запад"],
    "meters": ["", "1", "3", "5", "7", "10", "15", "20", "25", "35", "50", "75","100"],
    "degshours": ["", "15", "30", "45", "60", "75","90", "105", "120", "135", "150","165", "180", "195", "210","225", "240", "255", "270","285", "300", "315", "330","345", "360"],
    "object1": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "nearest": ["",  "nearest_from_type"],
    "relation1": ["нет_отношения", "около", "позади", "перед","слева от", "правее"],
    "object2": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "relation2": ["нет_отношения", "около", "позади", "перед","слева от", "правее"],
    "object3": ["ничего", "дом", "дерево", "сломанное дерево","лес", "яма", "человек", "холм",
                "трещина", "лежачий человек", "камень", "останец","барьер", "фонарь", "автомобиль", "circle", "route"],
    "self": ["", "behind_of", "in_front_of", "to_left_from","to_right_from"],
    "gaze": ["", "has_gaze_focus_on"]
}


In [4]:
df = pd.read_csv("../../Data/Raw/merged_with_labels_11_patterns.csv", sep=";")
df = df.loc[df["type"].isin(["students", "crowdsource"])]
df.drop_duplicates(inplace=True)

In [5]:
df["y"].value_counts()

8    995
7    130
1     85
5     53
0     48
Name: y, dtype: int64

In [6]:
df.loc[:,['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].sum(0)

action       7289
direction     195
meters          0
degshours       0
object1      4163
nearest         0
relation1    1700
object2      3900
relation2    1395
object3      3200
self            0
gaze            0
dtype: int64

In [7]:
df[df["y"]==5].loc[:,['action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze']].sum(0)

action       318
direction      0
meters         0
degshours      0
object1      183
nearest        0
relation1      0
object2        0
relation2      0
object3        0
self           0
gaze           0
dtype: int64

In [8]:
df.columns

Index(['x', 'y_name', 'y', 'action', 'direction', 'meters', 'degshours',
       'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3',
       'self', 'gaze', 'type'],
      dtype='object')

In [76]:
with open(OUTPUT_DIR / "simple_crowd.txt", "w") as f:
    for ir, row in df[df["y"]==0].iterrows():
        f.write("Атрибуты: действие={}".format(descriptions["action"][row["action"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")

In [81]:
with open(OUTPUT_DIR / "direction_crowd.txt", "w") as f:
    for ir, row in df[df["y"]==1].iterrows():
        f.write("Атрибуты: действие={}; направление={}".format(descriptions["action"][row["action"]],
                                                              descriptions["direction"][row["direction"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")

In [89]:
with open(OUTPUT_DIR / "move_to_object_crowd.txt", "w") as f:
    for ir, row in df[df["y"]==5].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}".format(descriptions["action"][row["action"]],
                                                              descriptions["object1"][row["object1"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")

In [95]:
with open(OUTPUT_DIR / "2objects_crowd.txt", "w") as f:
    for ir, row in df[df["y"]==7].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}; отношение1-2={}; объект№2={}".format(descriptions["action"][row["action"]],
                                                                                          descriptions["object1"][row["object1"]],
                                                                                          descriptions["relation1"][row["relation1"]],
                                                                                          descriptions["object2"][row["object2"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")

In [9]:
df[df["y"]==8]

,x,y_name,y,action,direction,meters,degshours,object1,nearest,relation1,object2,relation2,object3,self,gaze,type
35509,"Продвигайся к ребёнку у ели, что около лесного...",relation2,8,6,0,0,0,6,0,1,2,1,4,0,0,crowdsource
35510,"Езжай к мужчине рядом с дубом, который у лесоп...",relation2,8,6,0,0,0,6,0,1,2,1,4,0,0,crowdsource
35511,"Беги к мужчине позади рва, который за коттеджем.",relation2,8,6,0,0,0,6,0,2,5,2,1,0,0,crowdsource
35512,"Шагай к женщине за канавой, которая сзади избу...",relation2,8,6,0,0,0,6,0,2,5,2,1,0,0,crowdsource
35513,"Проследуй к ребёнку позади кювета, который за ...",relation2,8,6,0,0,0,6,0,2,5,2,1,0,0,crowdsource
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49457,"возле лесо стоит человек, за лесом есть повале...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49458,"Отправляйся к спиленному деревуза лесополосой,...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49459,"Шагай в сторону дерева, лежащего позади леса, ...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource
49460,"Рядом с человеком есть лес, за которым повален...",relation2,8,6,0,0,0,3,0,2,4,1,6,0,0,crowdsource


In [10]:
with open(OUTPUT_DIR / "3objects_crowd_1.txt", "w") as f:
    for ir, row in df[df["y"]==8].iloc[:200].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}; отношение о2-о3={}; объект№3={}".format(descriptions["action"][row["action"]],
                                                                                          descriptions["object1"][row["object1"]],
                                                                                          descriptions["relation1"][row["relation1"]],
                                                                                          descriptions["object2"][row["object2"]],
                                                                                          descriptions["relation2"][row["relation2"]],
                                                                                          descriptions["object3"][row["object3"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")
with open(OUTPUT_DIR / "3objects_crowd_2.txt", "w") as f:
    for ir, row in df[df["y"]==8].iloc[200:400].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}; отношение о2-о3={}; объект№3={}".format(descriptions["action"][row["action"]],
                                                                                          descriptions["object1"][row["object1"]],
                                                                                          descriptions["relation1"][row["relation1"]],
                                                                                          descriptions["object2"][row["object2"]],
                                                                                          descriptions["relation2"][row["relation2"]],
                                                                                          descriptions["object3"][row["object3"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")
with open(OUTPUT_DIR / "3objects_crowd_3.txt", "w") as f:
    for ir, row in df[df["y"]==8].iloc[400:600].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}; отношение о2-о3={}; объект№3={}".format(descriptions["action"][row["action"]],
                                                                                          descriptions["object1"][row["object1"]],
                                                                                          descriptions["relation1"][row["relation1"]],
                                                                                          descriptions["object2"][row["object2"]],
                                                                                          descriptions["relation2"][row["relation2"]],
                                                                                          descriptions["object3"][row["object3"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")
with open(OUTPUT_DIR / "3objects_crowd_4.txt", "w") as f:
    for ir, row in df[df["y"]==8].iloc[600:800].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}; отношение о2-о3={}; объект№3={}".format(descriptions["action"][row["action"]],
                                                                                          descriptions["object1"][row["object1"]],
                                                                                          descriptions["relation1"][row["relation1"]],
                                                                                          descriptions["object2"][row["object2"]],
                                                                                          descriptions["relation2"][row["relation2"]],
                                                                                          descriptions["object3"][row["object3"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")
with open(OUTPUT_DIR / "3objects_crowd_5.txt", "w") as f:
    for ir, row in df[df["y"]==8].iloc[800:1000].iterrows():
        f.write("Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}; отношение о2-о3={}; объект№3={}".format(descriptions["action"][row["action"]],
                                                                                          descriptions["object1"][row["object1"]],
                                                                                          descriptions["relation1"][row["relation1"]],
                                                                                          descriptions["object2"][row["object2"]],
                                                                                          descriptions["relation2"][row["relation2"]],
                                                                                          descriptions["object3"][row["object3"]]) + "\n")
        f.write("Текст: "+ row["x"] +"\n")

In [11]:
OUTPUT_DIR

PosixPath('/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject')

# Готовим размеченные данные

In [146]:
counter = 0
simple_commands = []
direction_commands = []
oneobj_commands = []
twoobj_commands = []
threeobj_commands = []
with open("../../Data/Raw/data_v5.jsonl", "r") as f:
    for line in f:
        line = json.loads(line)
        if line[2][0] in [0, 1] and sum(line[2][1:])==0:
            simple_commands.append(line)
        elif line[2][0] in [2,3] and (sum(line[2][2:])==0):
            direction_commands.append(line)
        elif line[2][0] in [6]:
            if line[2][6]==0:
                oneobj_commands.append(line)
            elif line[2][6]!=0 and line[2][9]==0:
                twoobj_commands.append(line)
            elif line[2][6]!=0:
                threeobj_commands.append(line)
len(simple_commands), len(direction_commands), len(oneobj_commands), len(twoobj_commands), len(threeobj_commands)

(34, 2606, 5173, 2021, 989)

In [147]:
simple_commands = simple_commands[:100]
direction_commands = direction_commands[:100]
obj_commands = oneobj_commands[:50] + twoobj_commands[:50] + threeobj_commands[:50]

In [148]:
typesystem_path = "/media/grartem/B7DB5B36121B73AA/Projects/RL_robots/CommandClassifier/Notebooks/Data/ForWebannoProject/typesystem.xml"
with open(typesystem_path, "rb") as f:
    typesystem = load_typesystem(f)

In [149]:
nertype = typesystem.get_type("de.tudarmstadt.ukp.dkpro.core.api.ner.type.NamedEntity")
MetaDataType = typesystem.get_type('de.tudarmstadt.ukp.dkpro.core.api.metadata.type.DocumentMetaData')
SentenceType = typesystem.get_type('de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence')
TokenType = typesystem.get_type('de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token')

In [165]:
cls_to_attribute = {
    "A": "action",
    "D": "direction",
    "M": "meters",
    "DH": "degshours",
    "O1": "object1",
    "O2": "object2",
    "O3": "object3",
    "N": "nearest",
    "R1": "relation1",
    "R2": "relation2",
    "S": "self",
    "G": "gaze"
}
casData = Cas(typesystem)
text = ""
for line in simple_commands:
    line_attributes = "Атрибуты: действие={}".format(descriptions["action"][line[2][0]]) + "\n"
    casData.add_annotation(SentenceType(begin=len(text), end=len(text)+ len(line_attributes)))
    text += line_attributes
    command_position =  len(text)
    last_pos = len(text)
    last_cls = "O"
    entity_start = 0
    for t_i, (token, cls) in enumerate(zip(line[0],line[1])):
        #casData.add_annotation(TokenType(begin=last_pos, end=last_pos + len(token)))
        if cls!=last_cls:
            if last_cls=="O":
                entity_start = last_pos
            else:
                casEntity = nertype(value = cls_to_attribute[last_cls])
                casEntity.sofa = casData.get_sofa().xmiID
                casEntity.begin = entity_start
                casEntity.end = last_pos-1
                casData.add_annotation(casEntity)
                entity_start = last_pos
                
        else:
            pass
        last_cls = cls
        text += token + " "
        last_pos = len(text)
    if last_cls!="O":
        casEntity = nertype(value = cls_to_attribute[last_cls])
        casEntity.sofa = casData.get_sofa().xmiID
        casEntity.begin = entity_start
        casEntity.end = last_pos-1
        casData.add_annotation(casEntity)
    text = text[:-1] + "\n"
    casData.add_annotation(SentenceType(begin=command_position, end=len(text)))
casData.sofa_string = text
serializer = CasXmiSerializer()
with open(OUTPUT_DIR / "simple_commands_generated.xmi", "wb") as f:
    serializer.serialize(f, casData)

In [166]:
cls_to_attribute = {
    "A": "action",
    "D": "direction",
    "M": "meters",
    "DH": "degshours",
    "O1": "object1",
    "O2": "object2",
    "O3": "object3",
    "N": "nearest",
    "R1": "relation1",
    "R2": "relation2",
    "S": "self",
    "G": "gaze"
}
casData = Cas(typesystem)
text = ""
for line in direction_commands:
    line_attributes = "Атрибуты: действие={}; направление={}".format(descriptions["action"][line[2][0]],
                                                              descriptions["direction"][line[2][1]]) + "\n"
    casData.add_annotation(SentenceType(begin=len(text), end=len(text)+ len(line_attributes)))
    text += line_attributes
    command_position =  len(text)
    last_pos = len(text)
    last_cls = "O"
    entity_start = 0
    for t_i, (token, cls) in enumerate(zip(line[0],line[1])):
        #casData.add_annotation(TokenType(begin=last_pos, end=last_pos + len(token)))
        if cls!=last_cls:
            if last_cls=="O":
                entity_start = last_pos
            else:
                casEntity = nertype(value = cls_to_attribute[last_cls])
                casEntity.sofa = casData.get_sofa().xmiID
                casEntity.begin = entity_start
                casEntity.end = last_pos-1
                casData.add_annotation(casEntity)
                entity_start = last_pos
                
        else:
            pass
        last_cls = cls
        text += token + " "
        last_pos = len(text)
    if last_cls!="O":
        casEntity = nertype(value = cls_to_attribute[last_cls])
        casEntity.sofa = casData.get_sofa().xmiID
        casEntity.begin = entity_start
        casEntity.end = last_pos-1
        casData.add_annotation(casEntity)
    text = text[:-1] + "\n"
    casData.add_annotation(SentenceType(begin=command_position, end=len(text)))
casData.sofa_string = text
serializer = CasXmiSerializer()
with open(OUTPUT_DIR / "move_direction_generated.xmi", "wb") as f:
    serializer.serialize(f, casData)

In [167]:
cls_to_attribute = {
    "A": "action",
    "D": "direction",
    "M": "meters",
    "DH": "degshours",
    "O1": "object1",
    "O2": "object2",
    "O3": "object3",
    "N": "nearest",
    "R1": "relation1",
    "R2": "relation2",
    "S": "self",
    "G": "gaze"
}
casData = Cas(typesystem)
text = ""
for line in obj_commands:
    if line[2][9]!=0:
        line_attributes = "Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}; отношение о2-о3={}; объект№3={}".format(descriptions["action"][line[2][0]],
                                                                                          descriptions["object1"][line[2][4]],
                                                                                          descriptions["relation1"][line[2][6]],
                                                                                          descriptions["object2"][line[2][7]],
                                                                                          descriptions["relation2"][line[2][8]],
                                                                                          descriptions["object3"][line[2][9]]) + "\n"
    elif line[2][7]!=0:
        line_attributes = "Атрибуты: действие={}; объект№1={}; отношение о1-о2={}; объект№2={}".format(descriptions["action"][line[2][0]],
                                                                                          descriptions["object1"][line[2][4]],
                                                                                          descriptions["relation1"][line[2][6]],
                                                                                          descriptions["object2"][line[2][7]]) + "\n"
    else:
        line_attributes = "Атрибуты: действие={}; объект№1={}".format(descriptions["action"][line[2][0]],
                                                                                          descriptions["object1"][line[2][4]]) + "\n"
                                                                                                                                        
    casData.add_annotation(SentenceType(begin=len(text), end=len(text)+ len(line_attributes)))
    text += line_attributes
    command_position =  len(text)
    last_pos = len(text)
    last_cls = "O"
    entity_start = 0
    for t_i, (token, cls) in enumerate(zip(line[0],line[1])):
        #casData.add_annotation(TokenType(begin=last_pos, end=last_pos + len(token)))
        if cls!=last_cls:
            if last_cls=="O":
                entity_start = last_pos
            else:
                casEntity = nertype(value = cls_to_attribute[last_cls])
                casEntity.sofa = casData.get_sofa().xmiID
                casEntity.begin = entity_start
                casEntity.end = last_pos-1
                casData.add_annotation(casEntity)
                entity_start = last_pos
                
        else:
            pass
        last_cls = cls
        text += token + " "
        last_pos = len(text)
    if last_cls!="O":
        casEntity = nertype(value = cls_to_attribute[last_cls])
        casEntity.sofa = casData.get_sofa().xmiID
        casEntity.begin = entity_start
        casEntity.end = last_pos-1
        casData.add_annotation(casEntity)
    text = text[:-1] + "\n"
    casData.add_annotation(SentenceType(begin=command_position, end=len(text)))
casData.sofa_string = text
serializer = CasXmiSerializer()
with open(OUTPUT_DIR / "move_to_objects_generated.xmi", "wb") as f:
    serializer.serialize(f, casData)